##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']

In [4]:
#varr = 'O2'
varr = 'AOU'
#varr = 'TEMP'
#varr = 'SALT'

In [5]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_Phi_components_{varr}_Eo_space_horizontal_mean_LME11_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:    (lme: 11, Y: 64, L: 122, z_t: 35)
Coordinates:
  * lme        (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y          (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L          (L) int64 1 2 3 4 5 6 7 8 9 ... 115 116 117 118 119 120 121 122
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
Data variables:
    Phi_AOU_0  (lme, Y, L, z_t) float64 ...
    Phi_AOU_1  (lme, Y, L, z_t) float64 ...
    Phi_AOU_2  (lme, Y, L, z_t) float64 ...

In [6]:
dsf = xr.Dataset()
for cc in range(3):
    print(cc)
    dsf[f'Phi_{varr}_{cc}'] = xr_reshape(ds[f'Phi_{varr}_{cc}'][:,:,2:,:], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

0
1
2


In [7]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:    (lme: 11, Y: 64, z_t: 35, year: 10)
Coordinates:
  * lme        (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y          (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
  * year       (year) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Phi_AOU_0  (lme, Y, year, z_t) float64 2.775e-08 5.169e-08 ... -3.811e-07
    Phi_AOU_1  (lme, Y, year, z_t) float64 7.007e-08 1.294e-07 ... -9.406e-07
    Phi_AOU_2  (lme, Y, year, z_t) float64 1.786e-07 3.267e-07 ... -2.322e-06

In [8]:
dsf.to_netcdf(f'{path}/DPLE_driftcorrected_Phi_components_{varr}_Eo_space_horizontal_mean_LME11_ens_mean_monthly_to_yearly.nc', mode='w')